# Modelo Booleanno de Recuperación de la Información

## Leer m documentos (corpus)

In [123]:
import fitz # open pdf
import re #regex
import unicodedata # use unicode


def clean_doc(raw_doc):
    # remove accents
    ## decompose unicode glyphs
    normalized_string = unicodedata.normalize('NFKD',  text)
    ## if a glyhp is compose, use its base form
    no_accent_string = ''.join([c for c in normalized_string if not unicodedata.combining(c)])
    # remove punctuation marks
    no_punctuation_string = re.sub(r'[^\w]+', ' ', no_accent_string)
    # strip text of document to only get the main content
    return no_punctuation_string.strip()

def terms(clean_document):
    return re.split(r'[^\w]+', clean_document)
    
def terms_unique(clean_document):
    splited_words = terms(clean_document)
    terms = []
    for w in splited_words:
        if w not in terms:
            terms.append(w)
    return terms

def freq_term_table(terms):
    freq_table = {}
    for term in terms:
        if term in freq_table:
            freq_table[term] += 1
        else
            freq_table[term] = 1
    return freq_table

def read_raw_doc(doc_name):
    doc = fitz.open(doc_name)
    text = []
    for page in doc:
        text.append(page.get_text())
    return ''.join(text)


## Generar diccionario de términos de todo el corpus

## Aplicar eliminación de palabras vacías (stopword)

## Aplicar una técnica de stemming para reducir el "Tamaño" de las palabras

## Obtener una matriz binaria de la presencia de los términos en cada documento de todo el corpus

## Diseñar una tabla hash que permita obtener mediante su función. llave -> valor <=> stem -> documentos donde aparece stem

## Leer la consulta booleana Q

### Aplicar stopword y stemming a la consulta Q

### Aplicar la notación postfijo para el procesamiento de recuperación de la consulta dada

## Presentar los nombre de los documentos obtenidos por Q